In [29]:
import pandas as pd
# xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
# kfold
from sklearn.model_selection import KFold
# gridsearch
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# matplotlib
import matplotlib.pyplot as plt
# datetime
import datetime as dt
# numpy
import numpy as np
import sys
sys.path.insert(1, 'C:/Users/berna/Desktop/compes/DATATHON UPC FME 2023/healthcare_challenge/')
from utils import *

In [30]:
# load train data
df_train = pd.read_csv('../data/consumo_no23.csv')
df_test = pd.read_csv('../data/consumo_23.csv')
df_train = preprocess_cosumo(df_train)
df_test = preprocess_cosumo(df_test)

In [28]:
df_train

,FECHAPEDIDO,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,STACKS_COMPRATS,CODIGO_B40558,CODIGO_B41691,CODIGO_C26183,CODIGO_C56207,...,DEPARTMENT_90,DEPARTMENT_91,DEPARTMENT_92,DEPARTMENT_93,DEPARTMENT_94,DEPARTMENT_95,DEPARTMENT_96,DEPARTMENT_97,DEPARTMENT_98,DEPARTMENT_99
0,2016-01-02,40,10,102.803729,411.214916,4,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-02,20,5,12.100000,48.400000,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-02,100,50,215.325000,430.650000,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017-01-02,300,300,792.000000,792.000000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017-01-02,80,10,53.888999,431.111992,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13353,2022-10-31,305,5,26.399925,1610.395425,61,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13354,2022-10-31,210,5,5.652900,237.421800,42,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13355,2022-10-31,20,5,412.500000,1650.000000,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13356,2022-10-31,20,5,412.500000,1650.000000,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# train_test_split
X = df_train.drop(['CANTIDADCOMPRA'], axis = 1)
y = df_train['CANTIDADCOMPRA']
folds = KFold(n_splits=10, shuffle=True, random_state=42)

In [19]:
# timer
def timer(start_time=None):
    if not start_time:
        start_time = dt.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((dt.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))



In [20]:
# gridsearch for xgboost
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    # 'colsample_bytree': [0.8, 0.9, 1.0],
    # 'gamma': [0, 0.1, 0.2],
    # 'min_child_weight': [1, 3, 5],
    # 'reg_alpha': [0, 0.1, 0.5],
    # 'reg_lambda': [0, 0.1, 0.5],
    # 'objective': ['reg:squarederror','reg:squaredlogerror','reg:linear']
}

In [27]:
X

,FECHAPEDIDO,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,STACKS_COMPRATS,CODIGO_B40558,CODIGO_B41691,CODIGO_C26183,CODIGO_C56207,CODIGO_E64488,...,DEPARTMENT_90,DEPARTMENT_91,DEPARTMENT_92,DEPARTMENT_93,DEPARTMENT_94,DEPARTMENT_95,DEPARTMENT_96,DEPARTMENT_97,DEPARTMENT_98,DEPARTMENT_99
0,2016-01-02,10,102.803729,411.214916,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-02,5,12.100000,48.400000,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-02,50,215.325000,430.650000,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017-01-02,300,792.000000,792.000000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017-01-02,10,53.888999,431.111992,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13353,2022-10-31,5,26.399925,1610.395425,61,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13354,2022-10-31,5,5.652900,237.421800,42,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13355,2022-10-31,5,412.500000,1650.000000,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13356,2022-10-31,5,412.500000,1650.000000,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# xgboost
model = xgb.XGBRegressor(nthread=10, objective='reg:squarederror', silent=True)
# grid_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid, cv=folds, n_jobs=10, verbose=2, scoring=('neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'))
grid_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=folds, n_jobs=10, verbose=2, scoring='neg_mean_squared_error')
# start_time = timer(None)
grid_search.fit(X, y)
# timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 10 candidates, totalling 100 fits


ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\berna\anaconda3\envs\torch\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\berna\anaconda3\envs\torch\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "c:\Users\berna\anaconda3\envs\torch\lib\site-packages\xgboost\sklearn.py", line 988, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "c:\Users\berna\anaconda3\envs\torch\lib\site-packages\xgboost\sklearn.py", line 448, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  File "c:\Users\berna\an

In [ ]:
# metrics
X_test = df_test.drop(['CANTIDADCOMPRA'], axis = 1)
y_test = df_test['CANTIDADCOMPRA']
preds = grid_search.predict(X_test)

mse = 

In [10]:
start_time = timer(None)
[8 for i in range(100000)]
timer(start_time) # timing ends here for "start_time" variable


AttributeError: module 'datetime' has no attribute 'now'